In [ ]:
path = spark.conf.get("bundle.sourcePath")
import sys

sys.path.append(path)

In [4]:
import dlt

from uv_dab.constants import VOLUME_PATH
from uv_dab.low_level_functions.utils import (
    add_current_timestampe_to_df,
    add_word_count_to_df,
)

In [ ]:
@dlt.create_table(
    comment="Read raw facts from volume",
)
def raw_facts():
    return (
        spark.readStream.format("cloudFiles")
        .option("cloudFiles.format", "json")
        .option("cloudFiles.inferColumnTypes", "true")
        .load(VOLUME_PATH)
        .transform(add_current_timestampe_to_df)
    )

In [ ]:
# Augment Bronze table with a new column
@dlt.create_table(comment="Augmented table with word count")
def silver_facts():
    return dlt.read_stream("raw_facts").transform(add_word_count_to_df, "fact")

In [ ]:
# Group table by timestamp and count the number of facts
@dlt.create_table(comment="Grouped Table")
def gold_grouped_data():
    return dlt.read_stream("silver_facts").groupBy("ingestion_timestamp").count()